In [21]:
import os
import json
import pandas as pd
import fire
'''
{baseline_tag: 
    { 
        eval_dataset:{ }
                        
    }
}
'''

root_result_path = '../finetune_70B_results'
train_dataset='all_train'
base_model = "meta-llama/Meta-Llama-3-70B"

labeling_models = ["meta-llama/Meta-Llama-3.1-8B-Instruct", "gpt-4o-mini", "mistralai/Mistral-7B-Instruct-v0.3"]

eval_dataset_lists = ['mmlu', 'truthfulqa', 'gsm', 'bbh', 'tydiqa']

all_results = {}  
cur_results = {}

# Load results from JSON files
for labeling_model in labeling_models:
    all_results[labeling_model] = {}
    cur_results[labeling_model] = {}
    if 'llama' in labeling_model:
        # baseline_tags=['random',  'completion', 'perplexity',  'knn', 'less', 'full', 'label-filtered', 'diversity-filtered',  'filtered', 'filtered-cured-0.4'] #baselines
        baseline_tags=['label-filtered', 'diversity-filtered',  'filtered', 'filtered-cured-0.4'] #baselines

    elif 'gpt' in labeling_model:
        baseline_tags = ['label-filtered', 'diversity-filtered', 'filtered-cured-0.5']
    elif 'mistral' in labeling_model:
        baseline_tags = ['label-filtered', 'diversity-filtered', 'filtered-cured-0.5']
    else:
        raise NotImplementedError

    for tag in baseline_tags:
        baseline_results = {}
        for eval_dataset in eval_dataset_lists:
            path = f'{root_result_path}/{labeling_model}/{train_dataset}/{eval_dataset}/{base_model}/{tag}/metrics.json'
            try:
                with open(path, 'r') as f:
                    json_file = json.load(f)
                baseline_results[eval_dataset] = json_file
            except FileNotFoundError:
                print(f"Failed to find the file at {path}")
                baseline_results[eval_dataset] = None
        
        all_results[labeling_model][tag] = baseline_results

    # Extract relevant metrics and store in a DataFrame
    for tag in baseline_tags:
        baseline_result = []
        for eval_dataset in eval_dataset_lists:
            if all_results[labeling_model][tag][eval_dataset] is None:
                value = 0
            else:
                if eval_dataset == 'mmlu':
                    value = round(all_results[labeling_model][tag][eval_dataset]['average_acc'] * 100, 1)
                elif eval_dataset == 'bbh':
                    value = round(all_results[labeling_model][tag][eval_dataset]['average_exact_match']* 100, 1)
                elif eval_dataset == 'tydiqa':
                    value = round(all_results[labeling_model][tag][eval_dataset]['average']['f1'], 1)
                elif eval_dataset == 'gsm':
                    value = round(all_results[labeling_model][tag][eval_dataset]['exact_match']* 100, 1)
                elif eval_dataset == 'truthfulqa':
                    # value = round(all_results[labeling_model][tag][eval_dataset]["truth-info acc"]* 100, 1)
                    value = round(all_results[labeling_model][tag][eval_dataset]["MC2"]* 100, 1)
                else:
                    print("unknown eval dat·aset!")


            baseline_result.append(value)
        cur_results[labeling_model][tag] = baseline_result

# 构建 DataFrame
df_results = pd.DataFrame(cur_results, columns=['Labeling Model', 'Baseline Tag', 'Evaluation Dataset', 'Value'])

# 将 `Evaluation Dataset` 列作为列名，重构 DataFrame
df_pivot = df_results.pivot(index=['Labeling Model', 'Baseline Tag'], columns='Evaluation Dataset', values='Value')

# 计算平均值
df_pivot['Average Acc'] = df_pivot.mean(axis=1).round(1)

# 确保显示完整表格
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print(df_pivot)


Empty DataFrame
Columns: [Average Acc]
Index: []


In [24]:
import os
import json
import pandas as pd

# 定义路径和输入数据
root_result_path = '../finetune_70B_results'
train_dataset = 'all_train'
base_model = "meta-llama/Meta-Llama-3-70B"

labeling_models = ["meta-llama/Meta-Llama-3.1-8B-Instruct", "gpt-4o-mini", "mistralai/Mistral-7B-Instruct-v0.3"]
eval_dataset_lists = ['mmlu', 'truthfulqa', 'gsm', 'bbh', 'tydiqa']

all_results = []  # 用于存储所有结果行

# 遍历模型和标签，加载 JSON 文件数据
for labeling_model in labeling_models:
    if 'llama' in labeling_model:
        baseline_tags = [ 'completion', 'perplexity', 'knn', 'random', 'less', 'label-filtered', 'diversity-filtered', 'filtered', 'filtered-cured-0.4']
    elif 'gpt' in labeling_model:
        baseline_tags = ['label-filtered', 'diversity-filtered', 'filtered', 'filtered-cured-0.5']
    elif 'mistral' in labeling_model:
        baseline_tags = ['label-filtered', 'diversity-filtered', 'filtered', 'filtered-cured-0.5']
    else:
        raise NotImplementedError

    for tag in baseline_tags:
        row = {'Labeling Model': labeling_model, 'Baseline Tag': tag}
        for eval_dataset in eval_dataset_lists:
            path = f'{root_result_path}/{labeling_model}/{train_dataset}/{eval_dataset}/{base_model}/{tag}/metrics.json'
            try:
                with open(path, 'r') as f:
                    json_file = json.load(f)
                # 提取不同数据集的指标值
                if eval_dataset == 'mmlu':
                    value = round(json_file.get('average_acc', 0) * 100, 1)
                elif eval_dataset == 'bbh':
                    value = round(json_file.get('average_exact_match', 0) * 100, 1)
                elif eval_dataset == 'tydiqa':
                    value = round(json_file.get('average', {}).get('f1', 0), 1)
                elif eval_dataset == 'gsm':
                    value = round(json_file.get('exact_match', 0) * 100, 1)
                elif eval_dataset == 'truthfulqa':
                    # value = round(json_file.get('truth-info acc', 0) * 100, 1)
                    value = round(json_file.get('MC2', 0) * 100, 1)

                else:
                    value = 0
            except FileNotFoundError:
                print(f"Failed to find the file at {path}")
                value = 0
            row[eval_dataset] = value
        all_results.append(row)

# 构建 DataFrame
df_results = pd.DataFrame(all_results)

# 计算每个行的平均准确率
df_results['Average Acc'] = df_results[eval_dataset_lists].mean(axis=1).round(1)

# 确保显示完整表格
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# 打印 DataFrame
print(df_results)


                           Labeling Model        Baseline Tag  mmlu  truthfulqa   gsm   bbh  tydiqa  Average Acc
0   meta-llama/Meta-Llama-3.1-8B-Instruct          completion  67.1        44.6  85.0  80.0    32.7         61.9
1   meta-llama/Meta-Llama-3.1-8B-Instruct          perplexity  67.1        45.7  82.5  79.4    31.9         61.3
2   meta-llama/Meta-Llama-3.1-8B-Instruct                 knn  66.1        46.8  79.5  81.2    34.5         61.6
3   meta-llama/Meta-Llama-3.1-8B-Instruct              random  66.9        45.3  82.5  79.4    30.3         60.9
4   meta-llama/Meta-Llama-3.1-8B-Instruct                less  66.6        43.9  83.5  81.0    34.1         61.8
5   meta-llama/Meta-Llama-3.1-8B-Instruct      label-filtered  66.4        44.9  85.0  78.7    31.4         61.3
6   meta-llama/Meta-Llama-3.1-8B-Instruct  diversity-filtered  66.6        45.1  87.0  80.0    31.4         62.0
7   meta-llama/Meta-Llama-3.1-8B-Instruct            filtered  66.2        46.8  84.0  79.2    3